In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
import transformers
from transformers import AutoTokenizer, AutoModel
import torch

In [4]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")

In [5]:
train = pd.read_csv('../datasets/small_dataset_train_bert_bert_cosign.csv')
# test= pd.read_csv('datasets/test_ask_fm.csv')
# train=train.dropna()

In [6]:
train
hate=train.head(1000)
non_h=train.tail(1000)
hate
non_h

,q_a,label
16572,going to do you if would be the most attractiv...,0
16573,what celebrity do you think would were the mos...,0
16574,what celebrity do you think might want the for...,0
16575,what celebrity all you think would be thus muc...,0
16576,what celebrity do humans think so love the one...,0
...,...,...
17567,what do folk to mind about staying a virgin ti...,0
17568,what do you guys thinking over keeping scared ...,0
17569,what makes you men thinking about stayed a vir...,0
17570,what still you fucking think about staying a v...,0


In [7]:
for i in range(len(hate)):
    print(hate['augmented_qa'][i], hate['label'][i])

KeyError: 'augmented_qa'

In [42]:
for i in range(143066, 143165):
    print(non_h['augmented_qa'][i], non_h['label'][i])

whtype at do you guys thinking type about sttype aying type a virgin till mtype arritype age . 0
whgroup at do you guys thinking group about stgroup aying group a virgin till mgroup arrigroup age . 0
what do you guys thinking about staying a virgo till marriage . 0
what do you guys thinking about staying a virgo the virgin till marriage . 0
what do you guys thinking about staying a pure till marriage . 0
what do you guys thinking about staying a vestal till marriage . 0
what do you guys thinking about staying a virginal till marriage . 0
what do you guys thinking about staying a virtuous till marriage . 0
what do you guys thinking about staying a virgin boulder clay marriage . 0
what do you guys thinking about staying a virgin public treasury marriage . 0
what do you guys thinking about staying a virgin trough marriage . 0
what do you guys thinking about staying a virgin cashbox marriage . 0
what do you guys thinking about staying a virgin money box marriage . 0
what do you guys thinki

In [50]:
f_sen=[]
label=[]
scores=0
for i in range(3000):

    #Sentences we want sentence embeddings for
    sentences = [train['original_qa'][i],train['augmented_qa'][i]]
    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embedding_n= sentence_embeddings.numpy()
    score=cosine_similarity([sentence_embedding_n[0]],sentence_embedding_n[1:])
    scores=scores+score[0][0]
    if score[0][0]>.9:
        f_sen.append(train['augmented_qa'][i])
        label.append(train['label'][i]) 
        


In [46]:
scores/5000

0.9296422765910626

In [8]:
# my_dict = {'q_a':f_sen,'label':label}
# df = pd.DataFrame(my_dict)
# df.to_csv('small_dataset_train_backtranslation_bert_cosign.csv',index=False)